In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"
from jax.lib import xla_bridge
print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN2DTopoV1_ONOFF_local
from training import new_two_stage_training
from parameters import grid_pars, filter_pars, stimuli_pars, sig_pars, ssn_pars, conn_pars_m, conn_pars_s, ssn_layer_pars, conv_pars, training_pars, loss_pars

jax backend cpu


In [2]:
# Calculate A by using the default find_A function of SSN2DTopoV1_ONOFF_local when A is undefined
ssn_mid = SSN2DTopoV1_ONOFF_local(
    ssn_pars=ssn_pars, # got c_E and c_I as new parameters
    grid_pars=grid_pars,
    conn_pars=conn_pars_m,
    filter_pars=filter_pars,
    J_2x2 = ssn_layer_pars.J_2x2_m,
    gE = ssn_layer_pars.gE_m, 
    gI = ssn_layer_pars.gE_s
)

ssn_pars.A = ssn_mid.A
if ssn_pars.phases == 4:
    ssn_pars.A2 = ssn_mid.A2

####################### TRAINING PARAMETERS #############################
constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)

In [3]:
'''import jax.numpy as np
import numpy
from jax import random

import model
import training_supp
from util import take_log

logJ_2x2_s = take_log(ssn_layer_pars.J_2x2_s)
logs_2x2 = np.log(ssn_layer_pars.s_2x2_s)
logJ_2x2_m = take_log(ssn_layer_pars.J_2x2_m)
logJ_2x2 = [logJ_2x2_m, logJ_2x2_s]
log_sigma_oris = np.log(ssn_layer_pars.sigma_oris)
ssn_layer_pars_dict = dict(logJ_2x2=logJ_2x2, kappa_pre=ssn_layer_pars.kappa_pre, kappa_post=ssn_layer_pars.kappa_post) # from two_new_stage...
logJ_2x2 = ssn_layer_pars_dict['logJ_2x2']
kappa_pre = ssn_layer_pars_dict['kappa_pre']
kappa_post = ssn_layer_pars_dict['kappa_post']
    
constant_ssn_pars["logs_2x2"] = logs_2x2
constant_ssn_pars["train_ori"] = stimuli_pars.ref_ori
constant_ssn_pars["log_sigma_oris"] = log_sigma_oris
constant_ssn_pars["f_E"] = ssn_layer_pars.f_E
constant_ssn_pars["f_I"] = ssn_layer_pars.f_I
constant_ssn_pars["c_E"] = ssn_layer_pars.c_E
constant_ssn_pars["c_I"] = ssn_layer_pars.c_I

c_E = constant_ssn_pars['c_E']
c_I = constant_ssn_pars['c_I']
f_E = constant_ssn_pars['f_E']
f_I = constant_ssn_pars['f_I']
log_sigma_oris = constant_ssn_pars['log_sigma_oris']

w_sig = sig_pars.w_sig
b_sig = sig_pars.b_sig
constant_ssn_pars["ssn_mid_ori_map"] = ssn_mid.ori_map
ssn_mid_ori_map = constant_ssn_pars['ssn_mid_ori_map']

logs_2x2 = constant_ssn_pars['logs_2x2']
ssn_pars = constant_ssn_pars['ssn_pars']
grid_pars = constant_ssn_pars['grid_pars']
conn_pars_m = constant_ssn_pars['conn_pars_m']
conn_pars_s =constant_ssn_pars['conn_pars_s']
gE_m =constant_ssn_pars['gE'][0]
gE_s =constant_ssn_pars['gE'][1]
gI_m = constant_ssn_pars['gI'][0]
gI_s = constant_ssn_pars['gI'][1]
filter_pars = constant_ssn_pars['filter_pars']
conv_pars = constant_ssn_pars['conv_pars']
loss_pars = constant_ssn_pars['loss_pars']

constant_ssn_pars["key"] = random.PRNGKey(numpy.random.randint(0, 10000))
constant_ssn_pars = training_supp.generate_noise(constant_ssn_pars, training_pars.sig_noise, training_pars.batch_size, length=w_sig.shape[0], noise_type="poisson")
noise_ref = constant_ssn_pars['noise_ref']
noise_target = constant_ssn_pars['noise_target']
noise_type = constant_ssn_pars['noise_type']
train_ori = constant_ssn_pars['train_ori']
data = training_supp.create_data(stimuli_pars, n_trials=training_pars.batch_size)

model.two_layer_model(logJ_2x2=logJ_2x2, logs_2x2=logs_2x2, c_E=c_E, c_I=c_I, f_E=f_E, f_I=f_I, w_sig=w_sig, b_sig=b_sig, sigma_oris=log_sigma_oris, kappa_pre=kappa_pre, kappa_post=kappa_post, ssn_mid_ori_map=ssn_mid_ori_map, ssn_sup_ori_map=ssn_mid_ori_map, train_data=data, ssn_pars=ssn_pars, grid_pars=grid_pars, conn_pars_m=conn_pars_m, conn_pars_s=conn_pars_s, gE_m=gE_m, gI_m=gI_m, gE_s=gE_s, gI_s=gI_s, filter_pars=filter_pars, conv_pars=conv_pars, loss_pars=loss_pars, noise_ref=noise_ref, noise_target=noise_target)
'''

'import jax.numpy as np\nimport numpy\nfrom jax import random\n\nimport model\nimport training_supp\nfrom util import take_log\n\nlogJ_2x2_s = take_log(ssn_layer_pars.J_2x2_s)\nlogs_2x2 = np.log(ssn_layer_pars.s_2x2_s)\nlogJ_2x2_m = take_log(ssn_layer_pars.J_2x2_m)\nlogJ_2x2 = [logJ_2x2_m, logJ_2x2_s]\nlog_sigma_oris = np.log(ssn_layer_pars.sigma_oris)\nssn_layer_pars_dict = dict(logJ_2x2=logJ_2x2, kappa_pre=ssn_layer_pars.kappa_pre, kappa_post=ssn_layer_pars.kappa_post) # from two_new_stage...\nlogJ_2x2 = ssn_layer_pars_dict[\'logJ_2x2\']\nkappa_pre = ssn_layer_pars_dict[\'kappa_pre\']\nkappa_post = ssn_layer_pars_dict[\'kappa_post\']\n    \nconstant_ssn_pars["logs_2x2"] = logs_2x2\nconstant_ssn_pars["train_ori"] = stimuli_pars.ref_ori\nconstant_ssn_pars["log_sigma_oris"] = log_sigma_oris\nconstant_ssn_pars["f_E"] = ssn_layer_pars.f_E\nconstant_ssn_pars["f_I"] = ssn_layer_pars.f_I\nconstant_ssn_pars["c_E"] = ssn_layer_pars.c_E\nconstant_ssn_pars["c_I"] = ssn_layer_pars.c_I\n\nc_E = co

In [4]:
########### TRAINING ############
([ssn_layer_pars_dict, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    extra_stop=2,
    ssn_ori_map=ssn_mid.ori_map
)

from training_supp import sep_exponentiate
print(sep_exponentiate(ssn_layer_pars_dict['logJ_2x2'][0]))
print(ssn_layer_pars.J_2x2_m)
print(sep_exponentiate(ssn_layer_pars_dict['logJ_2x2'][1]))
print(ssn_layer_pars.J_2x2_s)

Loading orientation map
#### NOT SAVING! ####
Training loss: 3.4372832775115967 ¦ Validation -- loss: 3.872570753097534, true accuracy: 0.4699999988079071, at epoch 0, (time 21.341131448745728, 0.40685033798217773), 
Training accuracy: 0.5600000023841858, all losses[[1.6296686 ]
 [0.40785038]
 [0.57385534]
 [0.825909  ]
 [0.        ]
 [3.4372833 ]]
Training loss: 3.699962615966797 ¦ Validation -- loss: 3.279947519302368, true accuracy: 0.5799999833106995, at epoch 1, (time 18.330333471298218, 0.5619125366210938), 
Training loss: 2.2161335945129395 ¦ Validation -- loss: 2.1909475326538086, true accuracy: 0.7599999904632568, at epoch 20, (time 1.68170166015625, 0.4160428047180176), 
Entering second stage at epoch 20
Training loss: 2.1901259422302246 ¦ Validation -- loss: 2.1988868713378906, true accuracy: 0.75, at epoch 1, (time 1.68170166015625, 1.8430020809173584)
Training accuracy: 0.7400000095367432, all losses[[8.4341758e-01]
 [4.0583763e-01]
 [5.7432824e-01]
 [3.6639914e-01]
 [1.43

In [5]:
import jax.numpy as jnp
import jax
from jax import numpy as np

vv = lambda x, y: jnp.vdot(x, y)  #  ([a], [a]) -> []
mv = jax.vmap(vv, (0, None), 0) 
v1 = np.array([1,2,3])
v2 = np.array([1,0,1])
print(vv(v1,v2))
m1 = np.array([[1,0,1],[2, 0, 1], [4, 4, 4], [1, 1, 1], [0, 0, 0]])
print(mv(m1,v1))


4
[ 4  5 24  6  0]
